# Interactive generation of graphene moiré lattices
Author: [Tobias A. de Jong](https://github.com/TAdeJong)

This notebook uses `numpy`, `dask` and `matplotlib` to easily generate moiré lattices of two hexagonal graphene lattices. Using `ipywidgets`, we create an interactive plot where the angle can be adopted.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import ipywidgets as widgets

import dask.array as da

from latticegeneration import hexlattice_gen
from dask.distributed import Client, LocalCluster

import colorcet as cc

In [2]:
# Create a dask cluster for parallel computation. Settings optimized to play nice with mybinder.org
cluster = LocalCluster(n_workers=1, threads_per_worker=4, memory_limit='2GB')  
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:33747 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


In [3]:
S = 250  # Half size of visualization in pixels. Optimized for interactive rendering on mybinder.
r_k = 0.2 
a_0 = 0.246  # Lattice constant of graphene in nm

In [4]:
def plot(theta=1.05, kappa=1., psi=0., xi=0.):
    """Make a plot of a two stacked lattices, where the top lattice 
    is rotated by theta degrees, and stretched by kappa in 
    the direction psi degrees from the x-axis.
    The total stack is rotated by 'xi' degrees.
    """
    fig, ax = plt.subplots(figsize=[10,10])
    iterated1 = 0.7*hexlattice_gen(r_k, xi, 2, S)
    iterated2 = hexlattice_gen(r_k, theta+xi, 2, S, kappa=kappa, psi=psi)
    data = (iterated1 + iterated2).compute()
    im = ax.imshow(data.T, cmap='cet_fire_r', extent=[-S*r_k*a_0, S*r_k*a_0, S*r_k*a_0, -S*r_k*a_0], 
                   vmax=np.quantile(data,0.9),
                   vmin=np.quantile(data,0.1),
                  )
    ax.set_xlabel('x (nm)')
    ax.set_ylabel('y (nm)')
    ax.set_title(f'$\\theta = {theta:.2f}^\\circ, \\kappa = {kappa:.2f}^\\circ, \\psi = {psi:.2f}^\\circ$')

widgets.interactive(plot, 
                    theta=widgets.FloatSlider(value=1.05, min=0, max=5,step=0.05, 
                                              continuous_update=False, description='twist  $\\theta$'),
                    kappa=widgets.FloatSlider(value=1.0, min=1, max=1.1,step=0.005, 
                                              continuous_update=False, readout_format='.3f', 
                                              description='strain $\\kappa$'),
                    psi=widgets.FloatSlider(value=0, min=-90, max=90,step=5, 
                                            continuous_update=False, readout_format='.0f', 
                                            description='strain angle $\\psi$'),
                    xi=widgets.FloatSlider(value=15, min=-90, max=90,step=5, 
                                           continuous_update=False, readout_format='.0f', 
                                           description='substrate angle $\\xi$'),
   )

interactive(children=(FloatSlider(value=1.05, continuous_update=False, description='twist  $\\theta$', max=5.0…